In [1]:
# imports
import os, rasterio
import time
from deepforest import main, utilities, get_data, preprocess 
from os.path import exists
from  PIL import Image, ImagePath
Image.MAX_IMAGE_PIXELS = None
import matplotlib.pyplot as plt
import pandas as pd

c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# get and check Image
image_path=r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\img\DQ_2022_3b.tif"
# "D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\PNG\DQ_2022_3b.png"
print(exists(image_path))

# annotations = ("D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\annotation\\deep-forest-training.csv")
# print(exists(annotations))

exImg=Image.open(image_path)
print(exImg.getbands())
print(exImg.size)
src = rasterio.open(image_path)

transform = src.transform 
crs = src.crs
print(crs)

In [ ]:
# get training bonding box and convert to the right format
ano_shp=r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\DQ-Vegetation-Analysis-1180\03_Data\model-training\20230121-Five-classes-data-v1\5classes-treesdata-start-December2022-1.shp"
annotations=utilities.shapefile_to_annotations(ano_shp, image_path,  savedir='.')

In [ ]:
# remove first column
annotations

In [ ]:

ano_csv=r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\DQ-Vegetation-Analysis-1180\03_Data\model-training\20230121-Five-classes-data-v1\1class-treesdata-start-December2022.csv"
annotations.to_csv(ano_csv)

In [ ]:
print(pd.DataFrame(pd.read_csv(ano_csv,index_col=0)))

In [ ]:
#Write csv to file and crop to patches
tmpdir = (r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\crop")
# annotations.to_csv("{}/example.csv".format(tmpdir), index=False)
annotations_file = preprocess.split_raster(path_to_raster=image_path,
                                           annotations_file=ano_csv,
                                           base_dir=tmpdir,
                                           patch_size=500,
                                           patch_overlap=0.25)

# Returns a 6 column pandas array

In [ ]:
annotations_file
del(annotations_file['Unnamed: 0'])
assert annotations_file.shape[1] == 6

In [ ]:
annotations_file.to_csv(r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\crop\DQ_2022_3b.csv",index=0)


In [ ]:
ano_split=(r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\crop\DQ_2022_3b.csv")
print(pd.DataFrame(pd.read_csv("D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\crop\\DQ_2022_3b.csv")))
exists(ano_split)

In [ ]:

ano_split
# os.path.dirname(ano_split)

In [ ]:
model = main.deepforest()
model.use_release()

# Example run with short training
model.config["train"]['epochs'] = 5
model.config['gpus'] = '-1' #move to GPU and use all the GPU resources
# model.config['cpu'] = 'accelerator'
model.config["save-snapshot"] = False
model.config["train"]["csv_file"] = ano_split
model.config["train"]["root_dir"] = os.path.dirname(ano_split)
model.config['num_worker']=16
# model.config['max_epochs']=10
model.create_trainer()



In [ ]:
start_time = time.time()
model.trainer.fit(model)
print(f"--- Training on CPU: {(time.time() - start_time):.2f} seconds ---")

In [ ]:
image_path

In [ ]:
# predict to the image
# boxes = model.predict_image(path=image_path, return_plot = True)
predicted_raster = model.predict_tile(image_path, return_plot = False, patch_size=500, patch_overlap=0.25)

In [ ]:
pd.DataFrame(predicted_raster)

In [ ]:
# transform to projected cooridinate system
r = rasterio.open(image_path)
transform = r.transform 
crs = r.crs
print(crs)

gdf = utilities.annotations_to_shapefile(predicted_raster, transform=transform, crs=crs)

In [ ]:
gdf.to_file('DQ-trained-500-5e-5c.shp')

In [ ]:
results = model.evaluate(ano_split, os.path.dirname(ano_split), iou_threshold = 0.45)

In [ ]:
print(results["box_recall"])
print(results["box_precision"])

In [ ]:
results["results"].head()